In [11]:
import json

import numpy as np

import pandas as pd

In [12]:
# Leggo il file di configurazione
with open('config/config.json') as f:
    config_data = json.load(f)

# Ottengo i percorsi dei file
file_path_vaccinazioni_covid = config_data['path_covid_19_france_vaccinations_age_sexe_dep']
file_path_andamento_dei_prezzi_al_consumo = config_data['path_economicref_france_indice_prix_consommation']
file_path_risultati_delle_elezioni_legislative_1_turno = config_data['path_elections_france_legislatives_2022_1er_tour_par_bureau_de_vote']
file_path_risultati_delle_elezioni_legislative_2_turno = config_data['path_elections_france_legislatives_2022_2nd_tour_par_bureau_de_vote']
file_path_risultati_delle_elezioni_presidenziali_1_turno = config_data['path_elections_france_presidentielles_2022_1er_tour_par_bureau_de_vote']
file_path_risultati_delle_elezioni_presidenziali_2_turno = config_data['path_elections_france_presidentielles_2022_2nd_tour_par_bureau_de_vote']
file_path_dati_sulla_disoccupazione = config_data['path_labouref_france_departement_quarter_jobseeker']
file_path_andamento_dei_prezzi_dei_carburanti = config_data['path_prix_des_carburants_j_1_001']

In [16]:
# Caricamento e cleaning dataset vaccinazioni_covid

df_vaccinazioni_covid = pd.read_csv(file_path_vaccinazioni_covid, sep=';')
df_vaccinazioni_covid.rename(columns={"Date": "data", "Nombre de doses n°1": "numero_prime_dosi", "Nombre cumulé de doses n°1":"numero_cumulativo_prime_dosi", "Type de variable":"tipo_variabile","Valeur de la variable":"valore_variabile","Libellé de la variable":"etichetta_variabile", "Nom Officiel Département":"nome_dipartimento", "Code Officiel Région":"codice_regione", "Nom Officiel Région":"nome_regione", "Code Officiel Département":"codice_dipartimento", "Nombre de vaccinations complètes (doses n°2)":"numero_seconde_dosi", "Nombre de personnes vaccinées avec une dose de rappel (3e dose)":"numero_terze_dosi", "Nombre de personnes vaccinées avec une deuxième dose de rappel (4e dose)":"numero_quarte_dosi", "Nombre de doses de rappel bivalent":"numero_dosi_bivalenti", "Nombre de personnes ayant reçu une troisième dose de rappel":"numero_persone_terza_dose", "Nombre cumulé de vaccinations complètes (doses n°2)":"numero_cumulativo_seconde dosi", "Nombre cumulé de personnes vaccinées avec une dose de rappel (3e dose)":"numero_cumulativo_terze_dosi", "Nombre cumulé de personnes vaccinées avec une deuxième dose de rappel (4e dose)":"numero_cumulativo_quarte_dosi", "Nombre cumulé de personnes vaccinées avec une dose de rappel bivalent":"numero_cumulativo_dosi_bivalenti", "Nombre cumulé de personnes ayant reçu une troisième dose de rappel":"numero_cumulativo_persone_terza_dose_richiamo", "Couverture vaccinale partielle (dose n°1)":"copertura_vaccinale_prima_dose", "Couverture vaccinale complète (dose n°2)":"copertura_vaccinale_seconda_dose", "Couverture vaccinale rappel/3e dose":"copertura_vaccinale_terza_dose", "Couverture vaccinale 2e rappel/4e dose":"copertura_vaccinale_quarta_dose", "Couverture vaccinale du rappel bivalent":"copertura_vaccinale_dose_bivalente", "Couverture vaccinale de la troisième dose de rappel":"copertura_vaccinale_terza_dose_richiamo"}, inplace=True)
df_vaccinazioni_covid.drop(['numero_persone_terza_dose','numero_cumulativo_persone_terza_dose_richiamo', 'copertura_vaccinale_terza_dose_richiamo'], axis=1, inplace=True)
new_ordered_columns = ["data","nome_regione","codice_regione","nome_dipartimento","codice_dipartimento","tipo_variabile","etichetta_variabile","valore_variabile","numero_prime_dosi","numero_seconde_dosi","numero_terze_dosi","numero_quarte_dosi","numero_cumulativo_prime_dosi","numero_cumulativo_seconde dosi","numero_cumulativo_terze_dosi","numero_cumulativo_quarte_dosi","numero_dosi_bivalenti","numero_cumulativo_dosi_bivalenti", "copertura_vaccinale_prima_dose", "copertura_vaccinale_seconda_dose", "copertura_vaccinale_terza_dose", "copertura_vaccinale_quarta_dose", "copertura_vaccinale_dose_bivalente"]
df_vaccinazioni_covid = df_vaccinazioni_covid.reindex(columns=new_ordered_columns)
df_vaccinazioni_covid.head(10)

,data,nome_regione,codice_regione,nome_dipartimento,codice_dipartimento,tipo_variabile,etichetta_variabile,valore_variabile,numero_prime_dosi,numero_seconde_dosi,...,numero_cumulativo_seconde dosi,numero_cumulativo_terze_dosi,numero_cumulativo_quarte_dosi,numero_dosi_bivalenti,numero_cumulativo_dosi_bivalenti,copertura_vaccinale_prima_dose,copertura_vaccinale_seconda_dose,copertura_vaccinale_terza_dose,copertura_vaccinale_quarta_dose,copertura_vaccinale_dose_bivalente
0,2021-02-06,Auvergne-Rhône-Alpes,84,Ain,1.0,Par tranche d’âge,0-4,4,0,0,...,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0
1,2021-04-09,Auvergne-Rhône-Alpes,84,Ain,1.0,Par tranche d’âge,0-4,4,0,0,...,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0
2,2021-06-27,Auvergne-Rhône-Alpes,84,Ain,1.0,Par tranche d’âge,0-4,4,0,0,...,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0
3,2021-06-29,Auvergne-Rhône-Alpes,84,Ain,1.0,Par tranche d’âge,0-4,4,0,0,...,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0
4,2021-07-15,Auvergne-Rhône-Alpes,84,Ain,1.0,Par tranche d’âge,0-4,4,0,0,...,1,0,0,0,0,0.0,0.0,0.0,0.0,0.0
5,2021-07-18,Auvergne-Rhône-Alpes,84,Ain,1.0,Par tranche d’âge,0-4,4,0,0,...,2,0,0,0,0,0.0,0.0,0.0,0.0,0.0
6,2021-08-24,Auvergne-Rhône-Alpes,84,Ain,1.0,Par tranche d’âge,0-4,4,0,0,...,2,0,0,0,0,0.0,0.0,0.0,0.0,0.0
7,2021-08-29,Auvergne-Rhône-Alpes,84,Ain,1.0,Par tranche d’âge,0-4,4,0,0,...,2,0,0,0,0,0.0,0.0,0.0,0.0,0.0
8,2021-10-07,Auvergne-Rhône-Alpes,84,Ain,1.0,Par tranche d’âge,0-4,4,0,0,...,2,0,0,0,0,0.0,0.0,0.0,0.0,0.0
9,2021-10-17,Auvergne-Rhône-Alpes,84,Ain,1.0,Par tranche d’âge,0-4,4,0,0,...,2,0,0,0,0,0.0,0.0,0.0,0.0,0.0


In [14]:
# Caricamento e cleaning dataset andamento_prezzi_consumo

df_andamento_prezzi_consumo = pd.read_csv(file_path_andamento_dei_prezzi_al_consumo, sep=';')
df_andamento_prezzi_consumo.rename(columns={"Période" : "periodo", "Indice" : "suggerimento", "Valeur": "valore","Code" : "codice","Type" : "tipo","Sous-type" : "sottotipo", "Base" : "base", "Population" : "popolazione", "Zone concernée" : "zona_interessata", "Code COICOP" : "codice_COICOP", "Désignation COICOP" : "designazione_COICOP", "Série arrêtée" : "serie_interrotta", "Référence INSEE" : "riferimento_INSEE"}, inplace=True)
df_andamento_prezzi_consumo.head(10)

,periodo,suggerimento,valore,codice,tipo,sottotipo,base,popolazione,zona_interessata,codice_COICOP,designazione_COICOP,serie_interrotta,riferimento_INSEE
0,2017-02,Indice des prix à la consommation - Base 2015 ...,101.67,A,Indice des prix à la consommation,Valeurs mensuelles,Base 2015,Ensemble des ménages,France,09.2.2,Instruments de musique et biens durables desti...,Non,1763723
1,2017-02,Indice des prix à la consommation - Base 2015 ...,88.77,A,Indice des prix à la consommation,Valeurs mensuelles,Base 2015,Ensemble des ménages,France,09.5.1.4,Services d'édition et livres numériques,Non,1763762
2,2017-02,Indice des prix à la consommation - Base 2015 ...,101.19,A,Indice des prix à la consommation,Valeurs mensuelles,Base 2015,Ensemble des ménages,France,11.2.0,Services d'hébergement,Non,1763789
3,2017-02,Indice des prix à la consommation - Base 2015 ...,102.12,A,Indice des prix à la consommation,Valeurs mensuelles,Base 2015,Ensemble des ménages,France,11.2.0.3,Autres services d'hébergement,Non,1763792
4,2017-02,Indice des prix à la consommation - Base 2015 ...,101.02,A,Indice des prix à la consommation,Valeurs mensuelles,Base 2015,Ensemble des ménages,France,12.7,Autres services n.c.a.,Non,1763828
5,2017-02,Indice des prix à la consommation - Base 2015 ...,102.04,A,Indice des prix à la consommation,Valeurs mensuelles,Base 2015,Ensemble des ménages,France métropolitaine,01.1.8.3,Chocolat,Non,1763922
6,2017-02,Indice des prix à la consommation - Base 2015 ...,99.54,A,Indice des prix à la consommation,Valeurs mensuelles,Base 2015,Ensemble des ménages,France métropolitaine,01.2,Boissons non alcoolisées,Non,1763932
7,2017-02,Indice des prix à la consommation - Base 2015 ...,87.80,A,Indice des prix à la consommation,Valeurs mensuelles,Base 2015,Ensemble des ménages,France métropolitaine,03.2.1,Chaussures diverses,Non,1763973
8,2017-02,Indice des prix à la consommation - Base 2015 ...,101.88,A,Indice des prix à la consommation,Valeurs mensuelles,Base 2015,Ensemble des ménages,France métropolitaine,04.4.4,Services divers liés au logement n.c.a.,Non,1763999
9,2017-02,Indice des prix à la consommation - Base 2015 ...,97.90,A,Indice des prix à la consommation,Valeurs mensuelles,Base 2015,Ensemble des ménages,France métropolitaine,05.2.0.2,Linge de lit et accessoires de literie,Non,1764030


In [15]:
# Caricamento e cleaning dataset risultati_elezioni_legislative_1_turno

df_risultati_elezioni_legislative_1_turno = pd.read_csv(file_path_risultati_delle_elezioni_legislative_1_turno, sep=';', dtype={"Code du département": str, "Code de la commune": str, "Code du b.vote": str})
df_risultati_elezioni_legislative_1_turno.rename(columns={"Code du département" : "codice_dipartimento", "Libellé du département" : "nome_dipartimento", "Code de la circonscription" : "codice_della_circoscrizione", "Libellé de la circonscription" : "nome_della_circoscrizione", "Code de la commune" : "codice_del_comune", "Libellé de la commune" : "nome_del_comune", "Code du b.vote" : "codice_del_seggio_elettorale", "Inscrits" : "iscritti", "Abstentions" : "astenuti", "% Abs/Ins" : "% astenuti/iscritti", "Votants" : "votanti", "% Vot/Ins" : "% votanti/iscritti", "Blancs" : "bianche", "% Blancs/Ins" : "% bianche/iscritti", "% Blancs/Vot" : "% bianche/votanti", "Nuls" : "nulle", "% Nuls/Ins" : "% nulle/iscritti", "% Nuls/Vot" : "% nulle/votanti", "Exprimés" : "espressi", "% Exp/Ins" : "% espressi/iscritti", "% Exp/Vot" : "% espressi/votanti", "N°Panneau" : "pannello_numero", "Sexe" : "genere", "Nom" : "cognome", "Prénom" : "nome", "Nuance" : "tendenza_politica", "Voix" : "voti", "% Voix/Ins" : "% voti/iscritti", "% Voix/Exp" : "% voti/iscritti", "Code Officiel EPCI" : "codice_ufficiale_EPCI", "Nom Officiel EPCI" : "nome_ufficiale_dello_EPCI", "Code Officiel Région" : "codice_ufficiale_regionale", "Nom Officiel Région" : "nome_ufficiale_regionale", "location" : "coordinate", "lib_du_b_vote" : "etichetta_del_seggio_elettorale", "scrutin_code" : "codice_scrutinio"}, inplace=True)
df_risultati_elezioni_legislative_1_turno.head(10)

,codice_dipartimento,nome_dipartimento,codice_della_circoscrizione,nome_della_circoscrizione,codice_del_comune,nome_del_comune,codice_del_seggio_elettorale,iscritti,astenuti,% astenuti/iscritti,...,voti,% voti/iscritti,% voti/iscritti,codice_ufficiale_EPCI,nome_ufficiale_dello_EPCI,codice_ufficiale_regionale,nome_ufficiale_regionale,coordinate,etichetta_del_seggio_elettorale,codice_scrutinio
0,13,Bouches-du-Rhône,12,12ème circonscription,13054,Marignane,17,935,547,58.50,...,2,0.21,"0,52",200054807.0,Métropole d'Aix-Marseille-Provence,93.0,Provence-Alpes-Côte d'Azur,"43.415837, 5.220936",PARC DES SPORTS DU BOLMON,LG2022
1,13,Bouches-du-Rhône,12,12ème circonscription,13054,Marignane,19,862,448,51.97,...,8,0.93,"1,99",200054807.0,Métropole d'Aix-Marseille-Provence,93.0,Provence-Alpes-Côte d'Azur,"43.415837, 5.220936",CENTRE SOCIO CULT J RAIMU LE JAI,LG2022
2,13,Bouches-du-Rhône,12,12ème circonscription,13054,Marignane,20,984,648,65.85,...,4,0.41,"1,23",200054807.0,Métropole d'Aix-Marseille-Provence,93.0,Provence-Alpes-Côte d'Azur,"43.415837, 5.220936",GYMNASE DU CARESTIER,LG2022
3,13,Bouches-du-Rhône,12,12ème circonscription,13054,Marignane,21,604,372,61.59,...,1,0.17,"0,43",200054807.0,Métropole d'Aix-Marseille-Provence,93.0,Provence-Alpes-Côte d'Azur,"43.415837, 5.220936",GYMNASE DU CARESTIER,LG2022
4,13,Bouches-du-Rhône,12,12ème circonscription,13054,Marignane,23,980,652,66.53,...,4,0.41,"1,25",200054807.0,Métropole d'Aix-Marseille-Provence,93.0,Provence-Alpes-Côte d'Azur,"43.415837, 5.220936",PARC DES SPORTS DU BOLMON,LG2022
5,13,Bouches-du-Rhône,1,1ère circonscription,13055,Marseille,1003,1230,730,59.35,...,9,0.73,"1,83",200054807.0,Métropole d'Aix-Marseille-Provence,93.0,Provence-Alpes-Côte d'Azur,NaN,NaN,LG2022
6,13,Bouches-du-Rhône,1,1ère circonscription,13055,Marseille,1004,952,552,57.98,...,4,0.42,"1,03",200054807.0,Métropole d'Aix-Marseille-Provence,93.0,Provence-Alpes-Côte d'Azur,NaN,NaN,LG2022
7,13,Bouches-du-Rhône,1,1ère circonscription,13055,Marseille,1076,1172,666,56.83,...,106,9.04,"21,50",200054807.0,Métropole d'Aix-Marseille-Provence,93.0,Provence-Alpes-Côte d'Azur,NaN,NaN,LG2022
8,13,Bouches-du-Rhône,1,1ère circonscription,13055,Marseille,1076,1172,666,56.83,...,3,0.26,"0,61",200054807.0,Métropole d'Aix-Marseille-Provence,93.0,Provence-Alpes-Côte d'Azur,NaN,NaN,LG2022
9,13,Bouches-du-Rhône,1,1ère circonscription,13055,Marseille,1079,940,571,60.74,...,10,1.06,"2,79",200054807.0,Métropole d'Aix-Marseille-Provence,93.0,Provence-Alpes-Côte d'Azur,NaN,NaN,LG2022


In [16]:
# Caricamento e cleaning dataset risultati_elezioni_legislative_2_turno

df_risultati_elezioni_legislative_2_turno = pd.read_csv(file_path_risultati_delle_elezioni_legislative_2_turno, sep=';', dtype={"Code du département": str, "Code de la commune": str, "Code du b.vote": str})
df_risultati_elezioni_legislative_2_turno.rename(columns={"Code du département" : "codice_dipartimento", "Libellé du département" : "nome_dipartimento", "Code de la circonscription" : "codice_della_circoscrizione", "Libellé de la circonscription" : "nome_della_circoscrizione", "Code de la commune" : "codice_del_comune", "Libellé de la commune" : "nome_del_comune", "Code du b.vote" : "codice_del_seggio_elettorale", "Inscrits" : "iscritti", "Abstentions" : "astenuti", "% Abs/Ins" : "% astenuti/iscritti", "Votants" : "votanti", "% Vot/Ins" : "% votanti/iscritti", "Blancs" : "bianche", "% Blancs/Ins" : "% bianche/iscritti", "% Blancs/Vot" : "% bianche/votanti", "Nuls" : "nulle", "% Nuls/Ins" : "% nulle/iscritti", "% Nuls/Vot" : "% nulle/votanti", "Exprimés" : "espressi", "% Exp/Ins" : "% espressi/iscritti", "% Exp/Vot" : "% espressi/votanti", "N°Panneau" : "pannello_numero", "Sexe" : "genere", "Nom" : "cognome", "Prénom" : "nome", "Nuance" : "tendenza_politica", "Voix" : "voti", "% Voix/Ins" : "% voti/iscritti", "% Voix/Exp" : "% voti/iscritti", "Code Officiel EPCI" : "codice_ufficiale_EPCI", "Nom Officiel EPCI" : "nome_ufficiale_dello_EPCI", "Code Officiel Région" : "codice_ufficiale_regionale", "Nom Officiel Région" : "nome_ufficiale_regionale", "location" : "coordinate", "lib_du_b_vote" : "etichetta_del_seggio_elettorale", "scrutin_code" : "codice_scrutinio"}, inplace=True)
df_risultati_elezioni_legislative_2_turno.head(10)

,codice_dipartimento,nome_dipartimento,codice_della_circoscrizione,nome_della_circoscrizione,codice_del_comune,nome_del_comune,codice_del_seggio_elettorale,iscritti,astenuti,% astenuti/iscritti,...,voti,% voti/iscritti,% voti/iscritti,codice_ufficiale_EPCI,nome_ufficiale_dello_EPCI,codice_ufficiale_regionale,nome_ufficiale_regionale,coordinate,Libellé du bureau de vote,codice_scrutinio
0,76,Seine-Maritime,2,2ème circonscription,76412,Martainville-Epreville,1,620,349,56.29,...,119,19.19,"47,22",200070449.0,CC Inter-Caux-Vexin,28.0,Normandie,"49.459799, 1.294886",BUREAU UNIQUE,LG2022
1,76,Seine-Maritime,10,10ème circonscription,76417,Maucomble,1,312,135,43.27,...,91,29.17,"56,17",200070068.0,CC Communauté Bray-Eawy,28.0,Normandie,NaN,NaN,LG2022
2,76,Seine-Maritime,10,10ème circonscription,76430,Mesnil-Follemprise,1,98,44,44.90,...,23,23.47,"48,94",200070068.0,CC Communauté Bray-Eawy,28.0,Normandie,"49.773247, 1.299788",MAIRIE DE MESNIL FOLLEMPRISE,LG2022
3,76,Seine-Maritime,5,5ème circonscription,76436,Le Mesnil-sous-Jumièges,1,501,257,51.30,...,111,22.16,"49,33",200023414.0,Métropole Rouen Normandie,28.0,Normandie,"49.411874, 0.857707",MAIRIE DU MESNIL SOUS JUMIEGES,LG2022
4,76,Seine-Maritime,1,1ère circonscription,76451,Mont-Saint-Aignan,14,1052,405,38.50,...,458,43.54,"74,59",200023414.0,Métropole Rouen Normandie,28.0,Normandie,"49.46335, 1.089179",MAISON DES ASSOCIATIONS,LG2022
5,76,Seine-Maritime,10,10ème circonscription,76461,Neufbosc,1,279,137,49.10,...,40,14.34,"28,57",200070068.0,CC Communauté Bray-Eawy,28.0,Normandie,"49.644771, 1.36479",MAIRIE DE NEUFBOSC,LG2022
6,76,Seine-Maritime,2,2ème circonscription,76475,Franqueville-Saint-Pierre,1,838,432,51.55,...,146,17.42,"39,25",200023414.0,Métropole Rouen Normandie,28.0,Normandie,"49.400083, 1.178974",PETIT POUCET,LG2022
7,76,Seine-Maritime,5,5ème circonscription,76476,Port-Jérôme-sur-Seine,6,512,302,58.98,...,83,16.21,"43,68",200010700.0,CA Caux Seine Agglo,28.0,Normandie,"49.505671, 0.600505",SALLE CHARLES PEGUY,LG2022
8,76,Seine-Maritime,7,7ème circonscription,76481,Octeville-sur-Mer,5,840,440,52.38,...,255,30.36,"67,11",200084952.0,CU Le Havre Seine Métropole,28.0,Normandie,"49.558463, 0.121508",BUREAU 5,LG2022
9,76,Seine-Maritime,5,5ème circonscription,76495,Pavilly,2,641,337,52.57,...,133,20.75,"47,50",247600646.0,CC Caux - Austreberthe,28.0,Normandie,"49.577214, 0.944576",LA DAME BLANCHE N 1 GRANDE SALLE,LG2022


In [17]:
# Caricamento e cleaning dataset risultati_elezioni_presidenziali_1_turno

df_risultati_elezioni_presidenziali_1_turno = pd.read_csv(file_path_risultati_delle_elezioni_presidenziali_1_turno, sep=';', dtype={"Code du département": str, "Code de la commune": str, "Code du b.vote": str})
df_risultati_elezioni_presidenziali_1_turno.rename(columns={"Code du département" : "codice_dipartimento", "Libellé du département" : "nome_dipartimento", "Code de la circonscription" : "codice_della_circoscrizione", "Libellé de la circonscription" : "nome_della_circoscrizione", "Code de la commune" : "codice_del_comune", "Libellé de la commune" : "nome_del_comune", "Code du b.vote" : "codice_del_seggio_elettorale", "Inscrits" : "iscritti", "Abstentions" : "astenuti", "% Abs/Ins" : "% astenuti/iscritti", "Votants" : "votanti", "% Vot/Ins" : "% votanti/iscritti", "Blancs" : "bianche", "% Blancs/Ins" : "% bianche/iscritti", "% Blancs/Vot" : "% bianche/votanti", "Nuls" : "nulle", "% Nuls/Ins" : "% nulle/iscritti", "% Nuls/Vot" : "% nulle/votanti", "Exprimés" : "espressi", "% Exp/Ins" : "% espressi/iscritti", "% Exp/Vot" : "% espressi/votanti", "N°Panneau" : "pannello_numero", "Sexe" : "genere", "Nom" : "cognome", "Prénom" : "nome", "Nuance" : "tendenza_politica", "Voix" : "voti", "% Voix/Ins" : "% voti/iscritti", "% Voix/Exp" : "% voti/iscritti", "Code Officiel EPCI" : "codice_ufficiale_EPCI", "Nom Officiel EPCI" : "nome_ufficiale_dello_EPCI", "Code Officiel Région" : "codice_ufficiale_regionale", "Nom Officiel Région" : "nome_ufficiale_regionale", "location" : "coordinate", "lib_du_b_vote" : "etichetta_del_seggio_elettorale", "scrutin_code" : "codice_scrutinio"}, inplace=True)
df_risultati_elezioni_presidenziali_1_turno.head(10)

,codice_dipartimento,nome_dipartimento,codice_della_circoscrizione,nome_della_circoscrizione,codice_del_comune,nome_del_comune,codice_del_seggio_elettorale,iscritti,astenuti,% astenuti/iscritti,...,voti,% voti/iscritti,% voti/iscritti,codice_ufficiale_EPCI,nome_ufficiale_dello_EPCI,codice_ufficiale_regionale,nome_ufficiale_regionale,codice_scrutinio,coordinate,etichetta_del_seggio_elettorale
0,01,Ain,4,4ème circonscription,01001,L'Abergement-Clémenciat,1,645,108,16.74,...,149,23.10,"28,65",200069193.0,CC de la Dombes,84.0,Auvergne-Rhône-Alpes,PR2022,"46.14943, 4.924647",BUREAU 1
1,01,Ain,5,5ème circonscription,01002,L'Abergement-de-Varey,1,213,38,17.84,...,41,19.25,"23,98",240100883.0,CC de la Plaine de l'Ain,84.0,Auvergne-Rhône-Alpes,PR2022,"46.007249, 5.423436",MAIRIE
2,01,Ain,5,5ème circonscription,01004,Ambérieu-en-Bugey,3,1213,246,20.28,...,227,18.71,"23,92",240100883.0,CC de la Plaine de l'Ain,84.0,Auvergne-Rhône-Alpes,PR2022,"45.959083, 5.36516",B3 CHATEAU DES ECHELLES
3,01,Ain,5,5ème circonscription,01004,Ambérieu-en-Bugey,8,1129,228,20.19,...,212,18.78,"23,93",240100883.0,CC de la Plaine de l'Ain,84.0,Auvergne-Rhône-Alpes,PR2022,"45.959083, 5.36516",B8 ECOLE MATERNELLE DE TIRET
4,01,Ain,5,5ème circonscription,01004,Ambérieu-en-Bugey,8,1129,228,20.19,...,48,4.25,"5,42",240100883.0,CC de la Plaine de l'Ain,84.0,Auvergne-Rhône-Alpes,PR2022,"45.959083, 5.36516",B8 ECOLE MATERNELLE DE TIRET
5,01,Ain,4,4ème circonscription,01005,Ambérieux-en-Dombes,1,1282,234,18.25,...,279,21.76,"27,14",200042497.0,CC Dombes Saône Vallée,84.0,Auvergne-Rhône-Alpes,PR2022,"45.99446, 4.905257",SALLE DU CONSEIL
6,01,Ain,2,2ème circonscription,01008,Ambutrix,1,554,80,14.44,...,71,12.82,"15,33",240100883.0,CC de la Plaine de l'Ain,84.0,Auvergne-Rhône-Alpes,PR2022,"45.939014, 5.336852",MAIRIE D AMBUTRIX
7,01,Ain,3,3ème circonscription,01009,Andert-et-Condon,1,279,48,17.20,...,1,0.36,"0,44",200040350.0,CC Bugey Sud,84.0,Auvergne-Rhône-Alpes,PR2022,"45.782502, 5.664004",BUREAU UNIQUE
8,01,Ain,3,3ème circonscription,01009,Andert-et-Condon,1,279,48,17.20,...,17,6.09,"7,46",200040350.0,CC Bugey Sud,84.0,Auvergne-Rhône-Alpes,PR2022,"45.782502, 5.664004",BUREAU UNIQUE
9,01,Ain,5,5ème circonscription,01012,Aranc,1,283,62,21.91,...,9,3.18,"4,09",200042935.0,CA Haut - Bugey Agglomération,84.0,Auvergne-Rhône-Alpes,PR2022,"46.003122, 5.508168",BUREAU UNIQUE


In [18]:
# Caricamento e cleaning dataset risultati_elezioni_presidenziali_2_turno

df_risultati_elezioni_presidenziali_2_turno = pd.read_csv(file_path_risultati_delle_elezioni_presidenziali_2_turno, sep=';', dtype={"Code du département": str, "Code de la commune": str, "Code du b.vote": str})
df_risultati_elezioni_presidenziali_2_turno.rename(columns={"Code du département" : "codice_dipartimento", "Libellé du département" : "nome_dipartimento", "Code de la circonscription" : "codice_della_circoscrizione", "Libellé de la circonscription" : "nome_della_circoscrizione", "Code de la commune" : "codice_del_comune", "Libellé de la commune" : "nome_del_comune", "Code du b.vote" : "codice_del_seggio_elettorale", "Inscrits" : "iscritti", "Abstentions" : "astenuti", "% Abs/Ins" : "% astenuti/iscritti", "Votants" : "votanti", "% Vot/Ins" : "% votanti/iscritti", "Blancs" : "bianche", "% Blancs/Ins" : "% bianche/iscritti", "% Blancs/Vot" : "% bianche/votanti", "Nuls" : "nulle", "% Nuls/Ins" : "% nulle/iscritti", "% Nuls/Vot" : "% nulle/votanti", "Exprimés" : "espressi", "% Exp/Ins" : "% espressi/iscritti", "% Exp/Vot" : "% espressi/votanti", "N°Panneau" : "pannello_numero", "Sexe" : "genere", "Nom" : "cognome", "Prénom" : "nome", "Nuance" : "tendenza_politica", "Voix" : "voti", "% Voix/Ins" : "% voti/iscritti", "% Voix/Exp" : "% voti/iscritti", "Code Officiel EPCI" : "codice_ufficiale_EPCI", "Nom Officiel EPCI" : "nome_ufficiale_dello_EPCI", "Code Officiel Région" : "codice_ufficiale_regionale", "Nom Officiel Région" : "nome_ufficiale_regionale", "location" : "coordinate", "lib_du_b_vote" : "etichetta_del_seggio_elettorale", "scrutin_code" : "codice_scrutinio"}, inplace=True)
df_risultati_elezioni_presidenziali_2_turno.head(10)

,codice_dipartimento,nome_dipartimento,codice_della_circoscrizione,nome_della_circoscrizione,codice_del_comune,nome_del_comune,codice_del_seggio_elettorale,iscritti,astenuti,% astenuti/iscritti,...,voti,% voti/iscritti,% voti/iscritti,codice_ufficiale_EPCI,nome_ufficiale_dello_EPCI,codice_ufficiale_regionale,nome_ufficiale_regionale,codice_scrutinio,coordinate,etichetta_del_seggio_elettorale
0,01,Ain,5,5ème circonscription,01004,Ambérieu-en-Bugey,1,1130,322,28.50,...,355,31.42,"49,65",240100883.0,CC de la Plaine de l'Ain,84.0,Auvergne-Rhône-Alpes,PR2022,"45.959083, 5.36516",B1 ESPACE 1500
1,01,Ain,5,5ème circonscription,01004,Ambérieu-en-Bugey,8,1129,250,22.14,...,429,38.00,"53,16",240100883.0,CC de la Plaine de l'Ain,84.0,Auvergne-Rhône-Alpes,PR2022,"45.959083, 5.36516",B8 ECOLE MATERNELLE DE TIRET
2,01,Ain,5,5ème circonscription,01007,Ambronay,2,892,173,19.39,...,381,42.71,"57,38",240100883.0,CC de la Plaine de l'Ain,84.0,Auvergne-Rhône-Alpes,PR2022,"46.011473, 5.362792",SALLE POLYVALENTE
3,01,Ain,4,4ème circonscription,01025,Bâgé-Dommartin,3,650,151,23.23,...,227,34.92,"49,46",200071371.0,CC Bresse et Saône,84.0,Auvergne-Rhône-Alpes,PR2022,"46.337596, 4.989068",EX N 1 DOMMARTIN
4,01,Ain,3,3ème circonscription,01033,Valserhône,2,1031,392,38.02,...,276,26.77,"48,51",240100891.0,CC du Pays Bellegardien (CCPB),84.0,Auvergne-Rhône-Alpes,PR2022,"46.128346, 5.809133",CENTRE JEAN MARINET
5,01,Ain,3,3ème circonscription,01033,Valserhône,11,699,196,28.04,...,211,30.19,"46,58",240100891.0,CC du Pays Bellegardien (CCPB),84.0,Auvergne-Rhône-Alpes,PR2022,"46.124032, 5.833335",SALLE DES FETES LANCRANS
6,01,Ain,5,5ème circonscription,01039,Béon,1,356,103,28.93,...,114,32.02,"48,51",200040350.0,CC Bugey Sud,84.0,Auvergne-Rhône-Alpes,PR2022,"45.854814, 5.750663",MAIRIE
7,01,Ain,1,1ère circonscription,01053,Bourg-en-Bresse,9,797,225,28.23,...,329,41.28,"63,15",200071751.0,CA du Bassin de Bourg-en-Bresse,84.0,Auvergne-Rhône-Alpes,PR2022,"46.206562, 5.241979",GR SCOLAIRE ALAGNIER PRIMAIRE
8,01,Ain,1,1ère circonscription,01057,Boz,1,392,90,22.96,...,146,37.24,"53,68",200071371.0,CC Bresse et Saône,84.0,Auvergne-Rhône-Alpes,PR2022,"46.406632, 4.911525",MAIRIE DE BOZ
9,01,Ain,4,4ème circonscription,01075,Chaleins,1,983,219,22.28,...,333,33.88,"46,90",200070118.0,CC Val de Saône Centre,84.0,Auvergne-Rhône-Alpes,PR2022,"46.035431, 4.80399",COMMUNE DE CHALEINS


In [15]:
# Caricamento e cleaning dataset disoccupazione

df_disoccupazione = pd.read_csv(file_path_dati_sulla_disoccupazione, sep=';')
df_disoccupazione.rename(columns={"Période (Trimestre)": "periodo_trimestre","Date":"data","Catégorie":"categoria","Tranche d'âge":"fascia_eta","Nb moyen demandeur emploi":"numero_medio_persone_cercanti_lavoro","Nb moyen demandeur emploi Homme":"numero_medio_uomini_cercanti_lavoro","Nb moyen demandeur emploi Femme":"numero_medio_donne_cercanti_lavoro","Différence Nb demandeur emploi trimestre précédent":"differenza_con_cercanti_lavoro_trimestre_prec","Nb moyen entrées":"numero_medio_entrate","Nb moyen sorties":"numero_medio_uscite","Nb moyen demandeur emploi longue durée":"numero_medio_persone_cercanti_lavoro_lungo_termine","Code Officiel Département":"codice_dipartimento","Code Officiel Région":"codice_regione","Nom Officiel Région":"nome_regione","Code Iso 3166-3 Zone":"Code Iso 3166-3 Zona","Nom Officiel Département":"nome_dipartimento"}, inplace=True)
new_ordered_columns = ["periodo_trimestre","data","nome_regione","codice_regione","nome_dipartimento","codice_dipartimento","categoria","fascia_eta", "numero_medio_persone_cercanti_lavoro","numero_medio_persone_cercanti_lavoro_lungo_termine","numero_medio_uomini_cercanti_lavoro","numero_medio_donne_cercanti_lavoro","differenza_con_cercanti_lavoro_trimestre_prec","numero_medio_entrate","numero_medio_uscite","Code Iso 3166-3 Zona"]
df_disoccupazione = df_disoccupazione.reindex(columns=new_ordered_columns)
df_disoccupazione.drop(['Code Iso 3166-3 Zona', 'periodo_trimestre'], axis=1, inplace=True)
df_disoccupazione.head(10)

,data,nome_regione,codice_regione,nome_dipartimento,codice_dipartimento,categoria,fascia_eta,numero_medio_persone_cercanti_lavoro,numero_medio_persone_cercanti_lavoro_lungo_termine,numero_medio_uomini_cercanti_lavoro,numero_medio_donne_cercanti_lavoro,differenza_con_cercanti_lavoro_trimestre_prec,numero_medio_entrate,numero_medio_uscite
0,2012-04,Île-de-France,11,Seine-et-Marne,77,B,Indifférent,8560,NaN,NaN,NaN,210.0,NaN,NaN
1,2014-04,Île-de-France,11,Seine-et-Marne,77,B,Indifférent,9810,NaN,NaN,NaN,10.0,NaN,NaN
2,2017-01,Île-de-France,11,Seine-et-Marne,77,B,Indifférent,10960,NaN,NaN,NaN,-80.0,NaN,NaN
3,2017-07,Île-de-France,11,Seine-et-Marne,77,B,Indifférent,11440,NaN,NaN,NaN,420.0,NaN,NaN
4,2009-04,Île-de-France,11,Yvelines,78,B,Indifférent,7050,NaN,NaN,NaN,340.0,NaN,NaN
5,2013-01,Île-de-France,11,Yvelines,78,B,Indifférent,9100,NaN,NaN,NaN,300.0,NaN,NaN
6,2000-04,Île-de-France,11,Essonne,91,B,Indifférent,4370,NaN,NaN,NaN,-230.0,NaN,NaN
7,2001-01,Île-de-France,11,Essonne,91,B,Indifférent,4430,NaN,NaN,NaN,-170.0,NaN,NaN
8,2011-01,Île-de-France,11,Essonne,91,B,Indifférent,6670,NaN,NaN,NaN,120.0,NaN,NaN
9,1999-04,Île-de-France,11,Hauts-de-Seine,92,B,Indifférent,6120,NaN,NaN,NaN,40.0,NaN,NaN


In [20]:
#df_andamento_prezzi_carburanti = pd.read_csv(file_path_andamento_dei_prezzi_dei_carburanti, sep=';')